<div class="span5 alert alert-info">

<h3>Final Report:</h3><br>
https://docs.google.com/presentation/d/1wFLdEuDiOHQqsLwcKy2eU9ZN_KULnhRl_2Hjw-O8S5I/edit?usp=sharing

<h3>Instructions:</h3><br>
https://docs.google.com/document/d/16rmjB9Bsx-_BXyEW55CYxBlGNHuc7IqsVh5hlcpH8X0/edit?usp=sharing
</div>

In [2]:
import pandas as pd

In [6]:
#Load data for use (local computer file sustten -> this kernel file)
df = pd.read_excel(r"C:\Users\eshab\Downloads\STAR Math_v2_Characteristics_deid.xlsx")

In [10]:
df.head()
df.columns

Index(['SchoolYear', 'StudentIdentifier_deid', 'DistrictName',
       'SchoolName_deid', 'LaunchDate', 'CompletedDate', 'CompletedDateLocal',
       'AssessmentNumber', 'ExtraTime', 'ScaledScore',
       'StandardErrorOfMeasurement', 'RaschScore', 'UnifiedScore',
       'GradeEquivalent', 'PercentileRank', 'NormalCurveEquivalent',
       'TotalCorrect', 'TotalPossible', 'StudentGrowthPercentileFallFall',
       'StudentGrowthPercentileFallSpring',
       'StudentGrowthPercentileFallWinter',
       'StudentGrowthPercentileSpringFall',
       'StudentGrowthPercentileSpringSpring',
       'StudentGrowthPercentileWinterSpring', 'ScreeningPeriodWindowName',
       'ScreeningWindowStartDate', 'ScreeningWindowEndDate',
       'RenaissanceBenchmarkCategoryName', 'RenaissanceBenchmarkCategoryLevel',
       'RenaissanceBenchmarkCategoryNumberOfLevels',
       'RenaissanceBenchmarkCategoryMinPercentileRank',
       'RenaissanceBenchmarkCategoryMaxPercentileRank',
       'StateBenchmarkAssessmentN

In [ ]:
#Relevant Columns:
# SchoolName_deid: Anonymized school name
# StudentIdentifier_deid: Anonymized student ID
# ScreeningPeriodWindowName: Indicates the assessment window (Fall, Winter, Spring)
# CompletedDate: To find the most recent Spring assessment
# CurrentSGP: Growth metric (Student Growth Percentile)
# StateBenchmarkCategoryName: Proficiency level (Level 1, 2, 3, 4, or At/Above Benchmark)

In [22]:
# Keep only Spring window
spring_df = df[df['ScreeningPeriodWindowName'] == 'Spring']

# Sort by student and assessment date (latest first)
spring_df_sorted = spring_df.sort_values(by=["StudentIdentifier_deid", "CompletedDate"], ascending=[True, False])

# Keep only the most recent Spring assessment per student
latest_spring_df = spring_df_sorted.drop_duplicates(subset="StudentIdentifier_deid", keep="first")
print(latest_spring_df)

      SchoolYear  StudentIdentifier_deid  \
28041  2021-2022                111269.0   
27897  2021-2022                111304.0   
27862  2021-2022                111656.0   
24191  2021-2022                111766.0   
13419  2021-2022                111941.0   
...          ...                     ...   
20913  2021-2022                999765.0   
11504  2021-2022                999792.0   
6668   2021-2022                999873.0   
1962   2021-2022                999952.0   
27224  2021-2022                999993.0   

                                    DistrictName       SchoolName_deid  \
28041  West Contra Costa Unified School District  Raspberry Elementary   
27897  West Contra Costa Unified School District       Pear Elementary   
27862  West Contra Costa Unified School District    Apricot Elementary   
24191  West Contra Costa Unified School District      Mango Elementary   
13419  West Contra Costa Unified School District         Pomelo Middle   
...                        

In [23]:
# Filter for students with Current SGP > or = 66
sgp_df = df[["StudentIdentifier_deid", "CurrentSGP"]].dropna().drop_duplicates(subset="StudentIdentifier_deid", keep="last")
merged_df = pd.merge(latest_spring_df, sgp_df, on="StudentIdentifier_deid", suffixes=('', '_from_all'))

print(merged_df)

     SchoolYear  StudentIdentifier_deid  \
0     2021-2022                111269.0   
1     2021-2022                111304.0   
2     2021-2022                111656.0   
3     2021-2022                111766.0   
4     2021-2022                111941.0   
...         ...                     ...   
5159  2021-2022                999765.0   
5160  2021-2022                999792.0   
5161  2021-2022                999873.0   
5162  2021-2022                999952.0   
5163  2021-2022                999993.0   

                                   DistrictName       SchoolName_deid  \
0     West Contra Costa Unified School District  Raspberry Elementary   
1     West Contra Costa Unified School District       Pear Elementary   
2     West Contra Costa Unified School District    Apricot Elementary   
3     West Contra Costa Unified School District      Mango Elementary   
4     West Contra Costa Unified School District         Pomelo Middle   
...                                         .

In [26]:
# Use merged SGP and Spring Benchmark to find high growth & low proficiency
high_growth = merged_df["CurrentSGP_from_all"] >= 66
low_proficiency = merged_df["StateBenchmarkCategoryName"].isin(["Level 1", "Level 2"])
filtered_df = merged_df[high_growth & low_proficiency]

print(filtered_df)

     SchoolYear  StudentIdentifier_deid  \
2     2021-2022                111656.0   
10    2021-2022                113883.0   
16    2021-2022                114737.0   
21    2021-2022                115816.0   
31    2021-2022                117549.0   
...         ...                     ...   
5138  2021-2022                997615.0   
5142  2021-2022                998215.0   
5145  2021-2022                998727.0   
5152  2021-2022                999313.0   
5159  2021-2022                999765.0   

                                   DistrictName         SchoolName_deid  \
2     West Contra Costa Unified School District      Apricot Elementary   
10    West Contra Costa Unified School District  Boysenberry Elementary   
16    West Contra Costa Unified School District     Mandarin Elementary   
21    West Contra Costa Unified School District     Mandarin Elementary   
31    West Contra Costa Unified School District    Jackfruit Elementary   
...                              

In [27]:
# To use in Power BI, save this filtered_df to computer locally
filtered_df.to_csv("high_growth_low_proficiency_students.csv", index=False)
print('saved!')

saved!


In [31]:
filtered_df.columns
filtered_df.head

<bound method NDFrame.head of      SchoolYear  StudentIdentifier_deid  \
2     2021-2022                111656.0   
10    2021-2022                113883.0   
16    2021-2022                114737.0   
21    2021-2022                115816.0   
31    2021-2022                117549.0   
...         ...                     ...   
5138  2021-2022                997615.0   
5142  2021-2022                998215.0   
5145  2021-2022                998727.0   
5152  2021-2022                999313.0   
5159  2021-2022                999765.0   

                                   DistrictName         SchoolName_deid  \
2     West Contra Costa Unified School District      Apricot Elementary   
10    West Contra Costa Unified School District  Boysenberry Elementary   
16    West Contra Costa Unified School District     Mandarin Elementary   
21    West Contra Costa Unified School District     Mandarin Elementary   
31    West Contra Costa Unified School District    Jackfruit Elementary   
...

In [13]:
filtered_df = pd.read_csv(r"C:\Users\eshab\Downloads\high_growth_low_proficiency_students.csv")

<div class="span5 alert alert-info">

<h3>Real Dataset Analysis starts here!</h3><br>

</div>

In [14]:
# Now group by schools and count students with higher sgp and low proficiency
school_counts = filtered_df["SchoolName_deid"].value_counts().reset_index()
school_counts.columns = ["School", "Student Count"]
print(school_counts)

                     School  Student Count
0        Current Elementary             51
1      Raspberry Elementary             39
2           Lime Elementary             38
3           Pear Elementary             36
4              Spinach High             35
5              Guava Middle             33
6      Tangerine Elementary             31
7       Mandarin Elementary             30
8           Plum Elementary             28
9     Blackberry Elementary             28
10             Mango Middle             28
11       Apricot Elementary             26
12         Apple Elementary             23
13          Kiwi Elementary             23
14            Pomelo Middle             22
15           Asparagus K-12             21
16                Kale High             21
17         Grape Elementary             20
18     Jackfruit Elementary             20
19   Boysenberry Elementary             20
20           Fig Elementary             19
21    Strawberry Elementary             18
22        B

In [8]:
filtered_df.columns

Index(['SchoolYear', 'StudentIdentifier_deid', 'DistrictName',
       'SchoolName_deid', 'LaunchDate', 'CompletedDate', 'CompletedDateLocal',
       'AssessmentNumber', 'ExtraTime', 'ScaledScore',
       'StandardErrorOfMeasurement', 'RaschScore', 'UnifiedScore',
       'GradeEquivalent', 'PercentileRank', 'NormalCurveEquivalent',
       'TotalCorrect', 'TotalPossible', 'StudentGrowthPercentileFallFall',
       'StudentGrowthPercentileFallSpring',
       'StudentGrowthPercentileFallWinter',
       'StudentGrowthPercentileSpringFall',
       'StudentGrowthPercentileSpringSpring',
       'StudentGrowthPercentileWinterSpring', 'ScreeningPeriodWindowName',
       'ScreeningWindowStartDate', 'ScreeningWindowEndDate',
       'RenaissanceBenchmarkCategoryName', 'RenaissanceBenchmarkCategoryLevel',
       'RenaissanceBenchmarkCategoryNumberOfLevels',
       'RenaissanceBenchmarkCategoryMinPercentileRank',
       'RenaissanceBenchmarkCategoryMaxPercentileRank',
       'StateBenchmarkAssessmentN

In [17]:
# Count students by proficiency level
proficiency_counts = filtered_df["StateBenchmarkCategoryName"].value_counts().reset_index()
proficiency_counts.columns = ["Proficiency Level", "Student Count"]
proficiency_counts

,Proficiency Level,Student Count
0,Level 2,446
1,Level 1,443


In [18]:
# Avg GSP by proficiency level
avg_sgp_by_proficiency = filtered_df.groupby("StateBenchmarkCategoryName")["CurrentSGP_from_all"].mean().reset_index()
avg_sgp_by_proficiency.columns = ["Proficiency Level", "Average SGP"]
avg_sgp_by_proficiency

,Proficiency Level,Average SGP
0,Level 1,78.882619
1,Level 2,82.307175


In [19]:
#Correlation between GSP and Each Benchmark Proficiency Level
from scipy.stats import spearmanr

# Spearman correlation for ordinal category level (Level 1 - Level 2)
correlation, p_value = spearmanr(filtered_df["CurrentSGP_from_all"], filtered_df["StateBenchmarkCategoryName"])
print(f"Spearman Correlation between SGP and Proficiency Level: {correlation:.2f}, p-value: {p_value:.4f}")


Spearman Correlation between SGP and Proficiency Level: 0.18, p-value: 0.0000


In [21]:
# Cross tabulation to show a table that quantifies how many students 
# fall into combinations of: high sgp and low sgp + low and high benchmark proficiencies

#Binning making tabulation cleaner
filtered_df["SGP_Bin"] = pd.cut(
    filtered_df["CurrentSGP_from_all"],
    bins=[66, 70, 75, 80, 85, 90, 95, 99],
    include_lowest=True,
    right=True
)

# Create the cross-tab: SGP bin vs. Proficiency Level
crosstab_sgp_benchmark = pd.crosstab(
    filtered_df["SGP_Bin"],
    filtered_df["StateBenchmarkCategoryName"]
)

# Show the results
print("\nCross-tabulation of SGP Bin vs State Benchmark Level:")
print(crosstab_sgp_benchmark)



Cross-tabulation of SGP Bin vs State Benchmark Level:
StateBenchmarkCategoryName  Level 1  Level 2
SGP_Bin                                     
(65.999, 70.0]                  100       60
(70.0, 75.0]                     80       79
(75.0, 80.0]                     84       67
(80.0, 85.0]                     68       61
(85.0, 90.0]                     51       72
(90.0, 95.0]                     43       61
(95.0, 99.0]                     17       46


In [23]:
# Group by GradeEquivalent and compute average SGP
cross_tab = filtered_df.groupby("GradeEquivalent")["CurrentSGP_from_all"].mean().reset_index()

# Rename columns for clarity
cross_tab.columns = ["GradeEquivalent", "Average_SGP"]

# Display the sorted cross-tabulated result
cross_tab.sort_values(by="GradeEquivalent")

,GradeEquivalent,Average_SGP
0,-1.0,73.0
1,1.5,89.0
2,1.6,75.0
3,1.7,67.0
4,1.8,71.0
...,...,...
88,12.2,96.0
89,12.5,69.5
90,12.8,87.0
91,12.9,81.0
